## Stein's Phenomenon

Stein (1956) found that if the dimension of data $p>=3$, then the MLE estimator $\hat{\mu_n}$ is inadmissible. This property is known as **_Stein's phenomenon_**.

We start with definition of inadmissible estimators.

&emsp;&emsp;**DEFINITION** (Inadmissible)

> An estimator $\hat{\mu_n}$ of the parameter $\mu$ is called **_inadmissible_** on $R^p$ with respect to the squared risk if there exists another estimator $\mu_n^*$ such that
$$E||\mu_n^*-\mu ||^2\leq E||\hat{\mu}_n-\mu||^2\quad\quad \textit{for all }\mu\in R^p,$$
and there exists $\mu_0\in R^p$ such that
$$E||\mu_n^*-\mu_0 ||^2 < E||\hat{\mu}_n-\mu_0||^2.$$

&emsp;&emsp;In this case, we also call that $\mu_n^*$ dominates $\hat{\mu}_n$ . Otherwise, the estimator $\hat{\mu_n}$ is called admissible. An estimator is admissible if it is not systematically outperformed, i.e. if there does not exist another estimator which displays less error for all the underlying unknown parameters.

According to the difinition, Stein's phenomenon can be desribed like:

&emsp;&emsp;For $p>=3$, there exists $\hat{\mu}$ such that $\forall\mu$,
$$E||\hat{\mu}_n-\mu ||^2 < E||\hat{\mu}^{MLE}_n-\mu_0||^2,$$
which makes MLE inadmissible.

A typical choice is the James-Stein estimator given by James-Stein (1961) for Gaussian distribution. To state formally,

&emsp;&emsp;**THEOREM**

> Suppose there is only one single observation $Y\thicksim N_p(\mu,I_p)$ (we want to estimate $\mu$). Then $\hat{\mu}^{MLE}=Y$. 
> Define
$$\hat{\mu}^{JS}_n=(1-\frac{(p-2)}{||Y||^2})Y,$$
> then
$$E_{\mu}||\hat{\mu}^{JS}-\mu||^2<E_{\mu}||Y-\mu||^2=E_{\mu}||\hat{\mu}^{MLE}-\mu||^2.$$

Here, we use Monte Carlo simulation to verify this.

For simplicity, we assume $\mu=e_1$, where $e_1$ is the basis vector in which only the first element is 1. Define the following function which takes dimension $p$ and number of simulation $nsim$ as inputs to calculate the Monte Carlo simulation results of James Stein estimator.

In [3]:
import numpy as np

In [46]:
def JamesSteinVSmle_MC(p,nsim):
    mean=np.append(1.0,np.zeros((1,p-1)))
    cov=np.identity(p)
    sample=np.random.multivariate_normal(mean, cov, nsim)
    #print sample
    
    # mle risk
    mle_err=sample-mean
    risk_mle=np.linalg.norm(mle_err,axis=1)
    risk_mle=np.mean(risk_mle)
    
    # js risk
    js_est=(np.linalg.norm(sample,axis=1))**(-1)*(p-2)
    #print js_est
    #print np.ones(1,p)
    js_est=np.ones(1,p)-js_est
    #print js_est

In [47]:
JamesSteinVSmle_MC(2,5)

TypeError: data type not understood